In [2]:
# !pip install polars geopy
import polars as pl
from geopy.distance import distance as geodesic_distance
from IPython.display import display, Markdown

def md(text):
    display(Markdown(text))

In [3]:
# CONST
MIN_STOPS = 3

itineraries_cols = [
    "legId",
    "startingAirport",
    "destinationAirport",
    "travelDuration",
    "totalFare",
    "totalTravelDistance",
    "segmentsArrivalAirportCode",
]
sample = pl.read_csv("itineraries_100k.csv", separator=",").select(
    itineraries_cols
)
# !wc -l itineraries.csv
total_rows = 82138754


def run_query(query: callable, *args, dataset_size=None):
    """
    Run a query on itineraries with args and
    dataset_size mb of memory
    """
    if dataset_size is not None:
        row_size = sample.estimated_size('mb') / sample.height
        dataset_rows = int(dataset_size / row_size)
        print(f"Rows in dataset: {dataset_rows} ({dataset_rows / total_rows * 100:.2f}%)")
        !head -n {dataset_rows} itineraries.csv > itineraries_tmp.csv
        file_name = "itineraries_tmp.csv"
    else:
        file_name = "itineraries.csv"
    res: pl.LazyFrame = query(
        pl.scan_csv(file_name, separator=",").select(
            itineraries_cols
        ),
        *args,
    )
    return res

def dump_query(query: callable, *args, dataset_size=None):
    run_query(query, *args, dataset_size=dataset_size).sink_csv(query.__name__ + ".csv")

def collect_query(query: callable, *args, dataset_size=None):
    return run_query(query, *args, dataset_size=dataset_size).collect()

def count_stops(segments: str):
    return len(segments.split("||")) - 1

def filter_by_stops(itineraries: pl.DataFrame, min_stops: int):
    """
    Filter itineraries by minimum number of stops
    """
    return itineraries.filter(
        pl.col("segmentsArrivalAirportCode").map_elements(count_stops, return_dtype=int) >= min_stops
    )

def remove_last_stop(segments: str):
    return ",".join(segments.split("||")[:-1]) or ""
def add_stops_codes(itineraries: pl.DataFrame):
    """
    Add a column with the list of stops codes
    """
    return itineraries.with_columns(
        pl.col("segmentsArrivalAirportCode")
        .map_elements(remove_last_stop, return_dtype=str)
        .alias("stopsAirportCode")
    )

In [4]:
md(f"""
- **Query 1**

Obtener el ID, trayecto, precio y escalas de vuelos con {MIN_STOPS} o más escalas.
""")


- **Query 1**

Obtener el ID, trayecto, precio y escalas de vuelos con 3 o más escalas.


In [5]:
def query1(itineraries: pl.DataFrame, min_stops: int = MIN_STOPS):
    print(f"Query 1: {min_stops} stops")
    return (
        itineraries.pipe(filter_by_stops, min_stops)
        .pipe(add_stops_codes)
        .select(
            [
                "legId",
                "startingAirport",
                "destinationAirport",
                "totalFare",
                "stopsAirportCode",
            ]
        )
    )

In [6]:
dump_query(query1, dataset_size=100)

Rows in dataset: 969861 (1.18%)


Query 1: 3 stops


In [7]:
md(
    f"""
- **Query 2**

Obtener el ID y trayecto de vuelos cuya deistancia total
sea mayor al doble de la distancia directa entre puntos
origen-destino.
"""
)


- **Query 2**

Obtener el ID y trayecto de vuelos cuya deistancia total
sea mayor al doble de la distancia directa entre puntos
origen-destino.


In [18]:
def query2(itineraries: pl.DataFrame):
    def calculate_distance(row: pl.Struct):
        lat_start = row["Latitude_start"]
        lon_start = row["Longitude_start"]
        lat_dest = row["Latitude_dest"]
        lon_dest = row["Longitude_dest"]
        return geodesic_distance(
            (lat_start, lon_start), (lat_dest, lon_dest)
        ).miles

    airports_tmp = pl.scan_csv("airports-codepublic.csv", separator=";").select(
        ["Airport Code", "Latitude", "Longitude"]
    )
    return (
        itineraries.join(
            airports_tmp,
            left_on="startingAirport",
            right_on="Airport Code",
            how="inner",
            suffix="_start",
        )
        .with_columns(
            pl.col("Latitude").alias("Latitude_start"),
            pl.col("Longitude").alias("Longitude_start"),
        )
        .join(
            airports_tmp,
            left_on="destinationAirport",
            right_on="Airport Code",
            how="inner",
            suffix="_dest",
        )
        .with_columns(
            pl.struct(
                [
                    "Latitude_start",
                    "Longitude_start",
                    "Latitude_dest",
                    "Longitude_dest",
                ]
            )
            .map_elements(calculate_distance, return_dtype=float)
            .alias("distance")
        )
        .filter(pl.col("distance") * 4 < pl.col("totalTravelDistance"))
        .select(["legId", "startingAirport", "destinationAirport"])
    )

In [20]:
dump_query(query2, dataset_size=50)

Rows in dataset: 484930 (0.59%)


In [15]:
md(f"""- Query 3

Obtener el ID, trayecto, escalas y duración para los 2 vuelos
de cada trayecto con menor duración entre todos los vuelos de
{MIN_STOPS} escalas o más.
""")

- Query 3

Obtener el ID, trayecto, escalas y duración para los 2 vuelos
de cada trayecto con menor duración entre todos los vuelos de
3 escalas o más.


In [16]:
def query3(itineraries: pl.DataFrame, min_stops: int = MIN_STOPS):
    return (
        itineraries.pipe(filter_by_stops, min_stops)
        .pipe(add_stops_codes)
        .select(  # try commenting this line
            [
                "legId",
                "startingAirport",
                "destinationAirport",
                "travelDuration",
                "stopsAirportCode",
            ]
        )
        .group_by(["startingAirport", "destinationAirport"])
        .agg([pl.min("travelDuration")])
    )

In [53]:
res = collect_query(query3, dataset_size=10)
res.write_csv("query3.csv")
res

startingAirport,destinationAirport,travelDuration
str,str,str


In [ ]:
md(f"""- Query 4

El precio promedio y máximo trayecto de los
vuelos con precio mayor a la media general de precios.
""")

In [22]:
def query4(itineraries: pl.DataFrame):
    return (
        itineraries.select(
            ["startingAirport", "destinationAirport", "totalFare"]
        )
        .filter(pl.col("totalFare") > pl.col("totalFare").mean())
        .group_by(["startingAirport", "destinationAirport"])
        .agg(
            pl.col("totalFare").mean().alias("mean_price"),
            pl.col("totalFare").max().alias("max_price"),
        )
    )

In [24]:
res = collect_query(query4, dataset_size=10)
res.write_csv("query4.csv")
res.head(5)

Rows in dataset: 96986 (0.12%)


startingAirport,destinationAirport,mean_price,max_price
str,str,f64,f64
"""ORD""","""SFO""",561.52433,2233.61
"""OAK""","""LAX""",555.759172,1075.51
"""ORD""","""PHL""",551.964416,1015.6
"""EWR""","""OAK""",612.419714,1241.6
"""DTW""","""LGA""",459.39375,648.61
